In [ ]:
import os
import cv2
import json
import numpy as np
from sklearn.cluster import KMeans
from skimage.color import rgb2lab, deltaE_cie76

# Set the paths for keyframes and output
keyframes_path = '/Users/huyenphung/Desktop/senior-project/keyframe_information/keyframe/L00'
output_file_path = '/Users/huyenphung/Desktop/senior-project/keyframe_information/color_metadata'
os.makedirs(output_file_path, exist_ok=True)

# Define a custom color palette
color_palette = {
    'black': [0, 0, 0],
    'blue': [0, 0, 255],
    'brown': [150, 75, 0],
    'green': [0, 128, 0],
    'grey': [128, 128, 128],
    'orange': [255, 165, 0],
    'pink': [255, 192, 203],
    'purple': [128, 0, 128],
    'red': [255, 0, 0],
    'white': [255, 255, 255],
    'yellow': [255, 255, 0]
}

# Convert the RGB color palette to LAB for more accurate comparison
palette_lab = {name: rgb2lab(np.uint8([[rgb]])) for name, rgb in color_palette.items()}

# Function to find the closest color name
def get_closest_color_name(r, g, b):
    color_rgb = np.uint8([[[(r, g, b)]]])
    color_lab = rgb2lab(color_rgb)
    min_distance = float('inf')
    closest_color = None

    for name, lab in palette_lab.items():
        distance = deltaE_cie76(color_lab, lab)
        if distance < min_distance:
            min_distance = distance
            closest_color = name
    return closest_color

# Function to extract dominant colors using KMeans
def extract_dominant_colors(image, num_clusters=5):
    # Use the actual size of the image
    image = image.reshape((-1, 3))  # Reshape image to a 2D array of pixels

    # Cluster colors with KMeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=0)
    kmeans.fit(image)
    dominant_colors = np.round(kmeans.cluster_centers_).astype(int)

    # Map the dominant colors to the closest color names from the custom palette
    color_context = [get_closest_color_name(*color) for color in dominant_colors]
    return color_context

# Process each image in the directory
for filename in os.listdir(keyframes_path):
    if filename.endswith('.jpg'):
        keyframe_path = os.path.join(keyframes_path, filename)

        # Load the image and convert it from BGR to RGB
        try:
            image = cv2.imread(keyframe_path)
            if image is None:
                raise ValueError(f"Unable to load image {keyframe_path}")
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        except Exception as e:
            print(f"Error reading image {keyframe_path}: {e}")
            continue

        # Extract dominant colors from the image
        dominant_colors = extract_dominant_colors(image)

        # Save the result to a JSON file
        output_path = os.path.join(output_file_path, f'{os.path.splitext(filename)[0]}.json')
        with open(output_path, 'w') as output_file:
            json.dump(dominant_colors, output_file)

        print(f"Saved color extraction result for {filename} to {output_path}")
